In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn import datasets
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing
from tensorflow.keras.layers import Dense
from tensorflow.keras import Sequential
from tensorflow.keras.optimizers import Adam


In [62]:
df = sns.load_dataset('titanic')

In [118]:
df

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,0,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,0,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,0,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,0,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,0,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,2,0,27.0,0,0,13.0000,S,Second,man,True,NaN,Southampton,no,True
887,1,1,0,19.0,0,0,30.0000,S,First,woman,False,B,Southampton,yes,True
888,0,3,0,NaN,1,2,23.4500,S,Third,woman,False,NaN,Southampton,no,False
889,1,1,0,26.0,0,0,30.0000,C,First,man,True,C,Cherbourg,yes,True


In [65]:
def fn( v ):
    if v == 'male':
        return '1'
    else :
        return '0'

In [66]:
df['sex'] = df["sex"].apply(fn)

#### fare(승선가격),성별 에 따른 생존유무를 예측하시오
- 53 남자
- w,b를 구하고, 정확도를 구하시오(케라스,텐서)

### keras

In [94]:
xdata = df.iloc[:,[2,6]]
ydata = df.iloc[:,[0]]

In [95]:
xdata

,sex,fare
0,1,7.2500
1,0,71.2833
2,0,7.9250
3,0,53.1000
4,1,8.0500
...,...,...
886,1,13.0000
887,0,30.0000
888,0,23.4500
889,1,30.0000


In [101]:
ydata

,survived
0,0
1,1
2,1
3,1
4,0
...,...
886,0
887,1
888,0
889,1


In [96]:
IO = Dense(units=1, input_dim=2,activation='sigmoid')
model = Sequential([IO])
model.compile(loss='binary_crossentropy',
              optimizer=Adam(0.1),metrics=['accuracy'])
h = model.fit(xdata,ydata,epochs=1000)

Train on 891 samples
Epoch 1/1000
891/891 [==============================] - 0s 82us/sample - loss: 4.3364 - acc: 0.4848
Epoch 2/1000
891/891 [==============================] - 0s 38us/sample - loss: 0.8134 - acc: 0.6835
Epoch 3/1000
891/891 [==============================] - 0s 38us/sample - loss: 0.5547 - acc: 0.7464
Epoch 4/1000
891/891 [==============================] - 0s 37us/sample - loss: 0.5439 - acc: 0.7688
Epoch 5/1000
891/891 [==============================] - 0s 33us/sample - loss: 0.6248 - acc: 0.7407
Epoch 6/1000
891/891 [==============================] - 0s 30us/sample - loss: 0.5545 - acc: 0.7722
Epoch 7/1000
891/891 [==============================] - 0s 37us/sample - loss: 0.5501 - acc: 0.7688
Epoch 8/1000
891/891 [==============================] - 0s 39us/sample - loss: 0.5633 - acc: 0.7632
Epoch 9/1000
891/891 [==============================] - 0s 36us/sample - loss: 0.5453 - acc: 0.7722
Epoch 10/1000
891/891 [==============================] - 0s 37us/sample - loss:

In [102]:
model.predict(np.array([[1,53]]))

array([[0.7637897]], dtype=float32)

In [103]:
model.predict_classes(np.array([[1,53]]))

array([[1]])

### Tensor

In [106]:
x = tf.placeholder(tf.float32, shape=[None,2])
y = tf.constant(ydata, tf.float32)
w = tf.Variable(tf.random_uniform([2,1]))
b = tf.Variable(tf.random_uniform([1]))
z = tf.matmul(x,w)+b
hx = tf.nn.sigmoid(z) 

cost_i = y*(-tf.log(hx))+(1-y)*(-tf.log(1-hx))
cost = tf.reduce_mean(cost_i)


optimizer = tf.train.AdamOptimizer( 0.1 )
train = optimizer.minimize( cost )
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)

predict = tf.cast(hx>0.5, dtype=tf.float32)
accuracy = tf.reduce_mean(tf.cast(tf.equal(predict,y),dtype=tf.float32))

##### 정규화

In [134]:
df1 = df[['sex','fare','survived']]

In [135]:
def fn( v ):
    if v == 'male':
        return '1'
    else :
        return '0'

In [143]:
df1['sex'] = df1["sex"].apply(fn)

C:\anaconda\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [144]:
scale = preprocessing.MinMaxScaler()
dataN = scale.fit_transform(df1)
dataN

array([[0.        , 0.01415106, 0.        ],
       [0.        , 0.13913574, 1.        ],
       [0.        , 0.01546857, 1.        ],
       ...,
       [0.        , 0.04577135, 0.        ],
       [0.        , 0.0585561 , 1.        ],
       [0.        , 0.01512699, 0.        ]])

In [145]:
xdata = dataN[:,:-1]
ydata = dataN[:,[-1]]

In [154]:
x = tf.placeholder(tf.float32, shape=[None,2])
y = tf.constant(ydata, tf.float32)
w = tf.Variable(tf.random_uniform([2,1]))
b = tf.Variable(tf.random_uniform([1]))
z = tf.matmul(x,w)+b
hx = tf.nn.sigmoid(z) 

cost_i = y*(-tf.log(hx))+(1-y)*(-tf.log(1-hx))
cost = tf.reduce_mean(cost_i)


optimizer = tf.train.AdamOptimizer( 0.0001 )
train = optimizer.minimize( cost )
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)

predict = tf.cast(hx>0.5, dtype=tf.float32)
accuracy = tf.reduce_mean(tf.cast(tf.equal(predict,y),dtype=tf.float32))

In [155]:
for i in range(10000):
    sess.run(train,{x:xdata})
    print(i,sess.run(cost,{x:xdata}))

0 0.7165164
1 0.71649957
2 0.7164828
3 0.7164661
4 0.7164493
5 0.71643245
6 0.7164157
7 0.71639884
8 0.71638215
9 0.71636546
10 0.7163487
11 0.71633196
12 0.71631515
13 0.7162985
14 0.7162817
15 0.716265
16 0.7162482
17 0.7162315
18 0.7162147
19 0.716198
20 0.7161813
21 0.71616465
22 0.7161479
23 0.7161311
24 0.71611446
25 0.7160976
26 0.7160808
27 0.7160641
28 0.7160475
29 0.7160308
30 0.716014
31 0.71599734
32 0.7159806
33 0.715964
34 0.71594733
35 0.71593064
36 0.715914
37 0.7158974
38 0.7158808
39 0.71586406
40 0.71584743
41 0.71583074
42 0.7158142
43 0.7157974
44 0.7157808
45 0.71576416
46 0.7157474
47 0.7157307
48 0.71571416
49 0.7156976
50 0.7156809
51 0.7156642
52 0.71564746
53 0.71563107
54 0.7156144
55 0.7155977
56 0.7155811
57 0.7155644
58 0.71554786
59 0.7155313
60 0.71551466
61 0.71549815
62 0.7154815
63 0.71546483
64 0.7154482
65 0.7154317
66 0.7154152
67 0.7153985
68 0.715382
69 0.7153654
70 0.7153489
71 0.7153323
72 0.7153158
73 0.71529907
74 0.7152826
75 0.715266
76 0.

In [156]:
sess.run(predict,{x:xdata})

array([[0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],

In [157]:
sess.run(accuracy, {x:xdata})

0.6251403

In [152]:
sess.run(w)

array([[0.06588507],
       [7.673158  ]], dtype=float32)

In [153]:
sess.run(b)

array([-0.9350968], dtype=float32)

In [158]:
a = sess.run(hx, {x: xdata})

In [159]:
sess.run(predict, {x: [[1,53]]})

array([[1.]], dtype=float32)